# Amplifier Agents: Task Delegation and Specialization

This notebook demonstrates how to use Amplifier's agent system for task delegation. We'll build working examples showing how to configure specialized agents, manage delegation, and implement common patterns.

## What Are Agents?

**Agents** (sub-agents) are named configuration overlays that define specialized agent personas. They enable:

- **Task specialization**: Different agents for different work types
- **Model selection**: Use appropriate models for task complexity
- **Tool restriction**: Control what sub-agents can do
- **Cost optimization**: Use cheaper models for simpler tasks
- **Parallel execution**: Multiple agents working simultaneously

Think of it as: **Agents are the team - each member has a specialized role.**

## Part 1: Building a Basic Agent Configuration

Let's start by creating a simple agent configuration and understanding how inheritance works.

In [ ]:
import json
from pprint import pprint
from typing import Any

# Parent session configuration (what the main agent has)
parent_config = {
    "session": {"orchestrator": "loop-streaming", "context": "context-simple"},
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5", "api_key": "${ANTHROPIC_API_KEY}"},
        }
    ],
    "tools": [
        {"module": "tool-filesystem"},
        {"module": "tool-bash"},
        {"module": "tool-task"},  # This enables delegation!
    ],
}

print("Parent Session Configuration:")
print("=" * 60)
pprint(parent_config)

print("\nKey points:")
print("  ✓ Main agent uses Claude Sonnet 4.5")
print("  ✓ Has filesystem, bash, and task tools")
print("  ✓ tool-task enables delegation to sub-agents")

In [ ]:
# Now let's add an agent that inherits and overrides
parent_with_agent = {
    **parent_config,
    "agents": {
        "bug-hunter": {
            "description": "Specialized debugging agent with stronger model",
            # Override provider to use stronger model
            "providers": [
                {
                    "module": "provider-anthropic",
                    "config": {
                        "model": "claude-opus-4-1",  # Upgraded!
                        "temperature": 0.1,  # More deterministic
                    },
                }
            ],
            # Override tools (remove task to prevent recursion)
            "tools": [
                {"module": "tool-filesystem"},
                {"module": "tool-bash"},
                # NO tool-task! Can't delegate further
            ],
            "system": {
                "instruction": """
You are a debugging specialist with deep expertise in finding root causes.

Your approach:
1. Read and analyze relevant code files
2. Identify patterns and potential issues
3. Run tests to reproduce bugs
4. Trace execution flow
5. Propose specific, actionable fixes

Be systematic, thorough, and precise.
"""
            },
        }
    },
}

print("\nAgent Configuration (bug-hunter):")
print("=" * 60)
print(json.dumps(parent_with_agent["agents"]["bug-hunter"], indent=2))

print("\n" + "=" * 60)
print("What bug-hunter inherits and overrides:\n")
print("Inherited from parent:")
print("  ✓ orchestrator: loop-streaming")
print("  ✓ context: context-simple")
print("\nOverridden:")
print("  ⚡ Model: claude-sonnet-4-5 → claude-opus-4-1 (stronger!)")
print("  ⚡ Temperature: default → 0.1 (more deterministic)")
print("  ⚡ Tools: removed tool-task (prevent recursion)")
print("  ⚡ System instruction: specialized for debugging")

## Part 2: Simulating Agent Delegation

Let's simulate how the delegation flow works when the main agent calls a sub-agent.

In [ ]:
# Simulating the delegation mechanism
class AgentDelegator:
    """Simulates how tool-task delegates to sub-agents."""

    def __init__(self, parent_config: dict[str, Any]):
        self.parent_config = parent_config
        self.agents = parent_config.get("agents", {})

    def get_agent_config(self, agent_name: str) -> dict[str, Any]:
        """Build agent configuration by merging parent and agent overrides."""
        if agent_name not in self.agents:
            raise ValueError(f"Agent '{agent_name}' not found")

        agent_def = self.agents[agent_name]

        # Start with parent session config
        agent_config = {"session": self.parent_config["session"].copy()}

        # Apply agent overrides
        if "providers" in agent_def:
            agent_config["providers"] = agent_def["providers"]
        else:
            agent_config["providers"] = self.parent_config["providers"]

        if "tools" in agent_def:
            agent_config["tools"] = agent_def["tools"]
        else:
            agent_config["tools"] = self.parent_config["tools"]

        if "hooks" in agent_def:
            agent_config["hooks"] = agent_def["hooks"]
        elif "hooks" in self.parent_config:
            agent_config["hooks"] = self.parent_config["hooks"]

        # System instruction
        if "system" in agent_def:
            agent_config["system"] = agent_def["system"]

        return agent_config

    async def spawn_agent(self, agent_name: str, task: str) -> str:
        """Simulate spawning a sub-agent."""
        print(f"\n🚀 Spawning agent: {agent_name}")
        print(f"   Task: {task}")

        # Get agent configuration
        agent_config = self.get_agent_config(agent_name)

        print("\n📋 Agent Configuration:")
        print(f"   Model: {agent_config['providers'][0]['config']['model']}")
        print(f"   Tools: {', '.join(t['module'] for t in agent_config['tools'])}")
        print(f"   Can delegate: {'tool-task' in [t['module'] for t in agent_config['tools']]}")

        # Simulate agent execution
        print("\n⚙️  Agent executing...")

        # Simulate result
        result = f"[{agent_name}] Completed task: {task}"

        print("\n✅ Agent completed")
        print(f"   Result: {result}")

        return result


# Demo: Delegation flow
print("Simulating Agent Delegation")
print("=" * 60)

delegator = AgentDelegator(parent_with_agent)

# Simulate main agent delegating to bug-hunter
result = await delegator.spawn_agent("bug-hunter", "Investigate crash in auth.py at line 42")

print("\n" + "=" * 60)
print("Flow Summary:")
print("  1. Main agent identifies debugging task")
print("  2. Main agent calls tool-task to spawn 'bug-hunter'")
print("  3. tool-task builds agent config (parent + overrides)")
print("  4. New session created with bug-hunter config")
print("  5. bug-hunter executes independently")
print("  6. Result returned to main agent")
print("  7. Main agent continues with result")

## Part 3: Multiple Specialized Agents

Let's build a complete agent team with different specializations.

In [ ]:
# Building a complete agent team
agent_team_config = {
    "session": {"orchestrator": "loop-streaming", "context": "context-simple"},
    "providers": [{"module": "provider-anthropic", "config": {"model": "claude-sonnet-4-5"}}],
    "tools": [{"module": "tool-filesystem"}, {"module": "tool-bash"}, {"module": "tool-web"}, {"module": "tool-task"}],
    "agents": {
        # 1. Quick scout for fast analysis
        "quick-scout": {
            "description": "Fast analysis agent using Haiku for simple tasks",
            "providers": [
                {
                    "module": "provider-anthropic",
                    "config": {
                        "model": "claude-haiku-3-5",  # Fastest, cheapest
                        "temperature": 0.5,
                    },
                }
            ],
            "tools": [{"module": "tool-filesystem"}],
            "system": {"instruction": "Quick analysis only. Be fast and concise. List obvious issues."},
        },
        # 2. Deep analyzer for complex reasoning
        "deep-analyzer": {
            "description": "Thorough analysis agent using Opus for complex tasks",
            "providers": [
                {
                    "module": "provider-anthropic",
                    "config": {
                        "model": "claude-opus-4-1",  # Strongest, most expensive
                        "temperature": 0.3,
                    },
                }
            ],
            "tools": [{"module": "tool-filesystem"}, {"module": "tool-bash"}],
            "system": {
                "instruction": """
Thorough analysis with deep reasoning. Consider:
- Edge cases and corner cases
- Performance implications
- Security concerns
- Maintainability issues
- Testing requirements
"""
            },
        },
        # 3. Code architect for system design
        "code-architect": {
            "description": "System design and architecture specialist",
            "providers": [{"module": "provider-anthropic", "config": {"model": "claude-opus-4-1", "temperature": 0.2}}],
            "tools": [{"module": "tool-filesystem"}],
            "system": {
                "instruction": """
You are a software architect. Focus on:
- Scalable design patterns
- Clean architecture principles
- Modularity and separation of concerns
- API design
- System boundaries
"""
            },
        },
        # 4. Test writer for comprehensive tests
        "test-writer": {
            "description": "Test generation specialist",
            "providers": [
                {"module": "provider-anthropic", "config": {"model": "claude-sonnet-4-5", "temperature": 0.4}}
            ],
            "tools": [{"module": "tool-filesystem"}, {"module": "tool-bash"}],
            "system": {
                "instruction": """
You are a test engineer. Write comprehensive tests:
- Unit tests for individual functions
- Integration tests for components
- Edge case coverage
- Error handling tests
- Follow testing best practices (AAA pattern)
"""
            },
        },
        # 5. Documentation writer
        "doc-writer": {
            "description": "Technical documentation specialist",
            "providers": [
                {"module": "provider-anthropic", "config": {"model": "claude-sonnet-4-5", "temperature": 0.6}}
            ],
            "tools": [
                {"module": "tool-filesystem"},
                {"module": "tool-web"},  # Can research
            ],
            "system": {
                "instruction": """
You are a technical writer. Create clear documentation:
- Write for developers
- Include code examples
- Explain complex concepts simply
- Use consistent formatting
- Add diagrams when helpful
"""
            },
        },
    },
}

print("Agent Team Configuration")
print("=" * 60)
print(f"\nTeam size: {len(agent_team_config['agents'])} specialized agents\n")

for agent_name, agent_config in agent_team_config["agents"].items():
    model = agent_config["providers"][0]["config"]["model"]
    temp = agent_config["providers"][0]["config"].get("temperature", "default")
    tools = [t["module"] for t in agent_config["tools"]]

    print(f"{agent_name}:")
    print(f"  Description: {agent_config['description']}")
    print(f"  Model: {model}")
    print(f"  Temperature: {temp}")
    print(f"  Tools: {', '.join(tools)}")
    print()

## Part 4: Cost Analysis - Model Selection Impact

In [ ]:
# Analyzing cost implications of different agent configurations

# Approximate costs per 1M tokens (as of 2025)
model_costs = {
    "claude-haiku-3-5": {"input": 0.25, "output": 1.25},
    "claude-sonnet-4-5": {"input": 3.00, "output": 15.00},
    "claude-opus-4-1": {"input": 15.00, "output": 75.00},
}


def calculate_task_cost(model: str, input_tokens: int, output_tokens: int) -> float:
    """Calculate cost for a task."""
    costs = model_costs[model]
    input_cost = (input_tokens / 1_000_000) * costs["input"]
    output_cost = (output_tokens / 1_000_000) * costs["output"]
    return input_cost + output_cost


# Simulate different task scenarios
scenarios = [
    {"task": "Quick file count", "input_tokens": 500, "output_tokens": 100, "recommended_agent": "quick-scout"},
    {"task": "Code review", "input_tokens": 5000, "output_tokens": 2000, "recommended_agent": "deep-analyzer"},
    {"task": "Architecture design", "input_tokens": 8000, "output_tokens": 3000, "recommended_agent": "code-architect"},
]

print("Cost Analysis: Model Selection Impact")
print("=" * 60)
print("\nModel Costs (per 1M tokens):\n")
for model, costs in model_costs.items():
    print(f"{model}:")
    print(f"  Input:  ${costs['input']:.2f}")
    print(f"  Output: ${costs['output']:.2f}")
print()

print("=" * 60)
print("Scenario Cost Comparison:\n")

for scenario in scenarios:
    print(f"Task: {scenario['task']}")
    print(f"Tokens: {scenario['input_tokens']} in, {scenario['output_tokens']} out")
    print(f"Recommended: {scenario['recommended_agent']}\n")

    # Calculate cost with each model
    costs = {}
    for model in model_costs:
        cost = calculate_task_cost(model, scenario["input_tokens"], scenario["output_tokens"])
        costs[model] = cost

    # Show comparison
    for model, cost in sorted(costs.items(), key=lambda x: x[1]):
        recommended = (
            " ← RECOMMENDED"
            if model in agent_team_config["agents"][scenario["recommended_agent"]]["providers"][0]["config"]["model"]
            else ""
        )
        print(f"  {model}: ${cost:.6f}{recommended}")

    # Show savings
    cheapest = min(costs.values())
    most_expensive = max(costs.values())
    savings = ((most_expensive - cheapest) / most_expensive) * 100
    print(f"\n  💰 Savings using appropriate model: {savings:.1f}%")
    print()

print("=" * 60)
print("Key Takeaway:")
print("  Use Haiku for simple tasks (60x cheaper than Opus!)")
print("  Use Sonnet for most tasks (5x cheaper than Opus)")
print("  Use Opus only when complexity justifies the cost")

## Part 5: Preventing Infinite Recursion

One of the most important aspects of agent configuration is preventing recursive delegation.

In [ ]:
# Demonstrating recursion prevention


def check_recursion_safety(config: dict[str, Any]) -> dict[str, Any]:
    """Analyze configuration for recursion risks."""
    report = {"main_can_delegate": False, "agents_can_delegate": {}, "risks": [], "recommendations": []}

    # Check main agent
    main_tools = [t["module"] for t in config.get("tools", [])]
    report["main_can_delegate"] = "tool-task" in main_tools

    # Check each agent
    for agent_name, agent_config in config.get("agents", {}).items():
        agent_tools = [t["module"] for t in agent_config.get("tools", [])]
        can_delegate = "tool-task" in agent_tools
        report["agents_can_delegate"][agent_name] = can_delegate

        if can_delegate:
            report["risks"].append(f"⚠️  {agent_name} has tool-task (can delegate)")

    # Generate recommendations
    if not report["main_can_delegate"]:
        report["recommendations"].append("❌ Main agent needs tool-task to delegate")
    else:
        report["recommendations"].append("✅ Main agent can delegate")

    if report["risks"]:
        report["recommendations"].append("❌ Remove tool-task from sub-agents to prevent recursion")
    else:
        report["recommendations"].append("✅ Sub-agents cannot delegate (safe)")

    return report


# Test with safe configuration
print("Recursion Safety Analysis")
print("=" * 60)
print("\nTest 1: Safe Configuration (current agent_team_config)\n")

report = check_recursion_safety(agent_team_config)

print(f"Main agent can delegate: {report['main_can_delegate']}")
print("\nAgent delegation abilities:")
for agent, can_delegate in report["agents_can_delegate"].items():
    status = "⚠️  CAN delegate" if can_delegate else "✅ Cannot delegate"
    print(f"  {agent}: {status}")

print(f"\nRisks: {len(report['risks'])}")
for risk in report["risks"]:
    print(f"  {risk}")

print("\nRecommendations:")
for rec in report["recommendations"]:
    print(f"  {rec}")

# Test with unsafe configuration
print("\n" + "=" * 60)
print("Test 2: Unsafe Configuration (agents with tool-task)\n")

unsafe_config = {
    "tools": [{"module": "tool-filesystem"}, {"module": "tool-task"}],
    "agents": {
        "dangerous-agent": {
            "tools": [
                {"module": "tool-filesystem"},
                {"module": "tool-task"},  # ⚠️  BAD! Can delegate
            ]
        }
    },
}

report2 = check_recursion_safety(unsafe_config)

print(f"Risks: {len(report2['risks'])}")
for risk in report2["risks"]:
    print(f"  {risk}")

print("\nRecommendations:")
for rec in report2["recommendations"]:
    print(f"  {rec}")

print("\n" + "=" * 60)
print("Recursion Flow Visualization:\n")
print("UNSAFE (infinite):")
print("  User → Main → Agent-1 → Agent-2 → Agent-3 → ...")
print("                   ↑                              │")
print("                   └──────────────────────────────┘")
print("\nSAFE (terminates):")
print("  User → Main → Agent (completes task, returns)")
print("          ↑                           │")
print("          └───────────────────────────┘")

## Part 6: Configuration Inheritance Deep Dive

In [ ]:
# Deep dive into how configuration inheritance works
from copy import deepcopy


def merge_agent_config(parent: dict, agent_overrides: dict) -> dict:
    """Simulate how agent config is merged with parent."""
    result = {}

    # Session: always inherited
    result["session"] = parent.get("session", {}).copy()

    # Providers: override or inherit
    if "providers" in agent_overrides:
        result["providers"] = deepcopy(agent_overrides["providers"])
    else:
        result["providers"] = deepcopy(parent.get("providers", []))

    # Tools: override or inherit
    if "tools" in agent_overrides:
        result["tools"] = deepcopy(agent_overrides["tools"])
    else:
        result["tools"] = deepcopy(parent.get("tools", []))

    # Hooks: override or inherit
    if "hooks" in agent_overrides:
        result["hooks"] = deepcopy(agent_overrides["hooks"])
    elif "hooks" in parent:
        result["hooks"] = deepcopy(parent["hooks"])

    # System instruction: override only
    if "system" in agent_overrides:
        result["system"] = agent_overrides["system"]

    return result


# Example: Show inheritance in action
print("Configuration Inheritance Deep Dive")
print("=" * 60)

# Parent config
parent = {
    "session": {"orchestrator": "loop-streaming", "context": "context-persistent"},
    "providers": [{"module": "provider-anthropic", "config": {"model": "claude-sonnet-4-5", "max_tokens": 4096}}],
    "tools": [{"module": "tool-filesystem"}, {"module": "tool-bash"}, {"module": "tool-task"}],
    "hooks": [{"module": "hooks-logging", "config": {"level": "INFO"}}],
}

# Agent with minimal overrides
minimal_agent = {"system": {"instruction": "You are a minimal agent."}}

# Agent with full overrides
full_override_agent = {
    "providers": [{"module": "provider-anthropic", "config": {"model": "claude-opus-4-1", "temperature": 0.1}}],
    "tools": [{"module": "tool-filesystem"}],
    "hooks": [{"module": "hooks-logging", "config": {"level": "DEBUG"}}],
    "system": {"instruction": "You are a fully customized agent."},
}

print("\nParent Configuration:")
print(json.dumps(parent, indent=2))

print("\n" + "=" * 60)
print("Example 1: Minimal Override Agent\n")
print("Agent Override:")
print(json.dumps(minimal_agent, indent=2))

result1 = merge_agent_config(parent, minimal_agent)
print("\nResulting Configuration:")
print(json.dumps(result1, indent=2))
print("\nNote: Everything inherited except system instruction")

print("\n" + "=" * 60)
print("Example 2: Full Override Agent\n")
print("Agent Override:")
print(json.dumps(full_override_agent, indent=2))

result2 = merge_agent_config(parent, full_override_agent)
print("\nResulting Configuration:")
print(json.dumps(result2, indent=2))
print("\nNote: Session still inherited, everything else overridden")

print("\n" + "=" * 60)
print("Inheritance Rules Summary:\n")
print("  • session: ALWAYS inherited (cannot override)")
print("  • providers: Override OR inherit (all-or-nothing)")
print("  • tools: Override OR inherit (all-or-nothing)")
print("  • hooks: Override OR inherit (all-or-nothing)")
print("  • system: Override only (no inheritance)")

## Part 7: Common Delegation Patterns

In [ ]:
# Demonstrating common delegation patterns

print("Common Agent Delegation Patterns")
print("=" * 60)

# Pattern 1: Progressive Delegation
print("\nPattern 1: Progressive Delegation (Scout → Specialist)\n")
print("Use case: Start cheap, escalate if needed\n")

progressive_pattern = {
    "agents": {
        "quick-scout": {
            "description": "Fast initial check (Haiku - $0.25/1M)",
            "providers": [{"module": "provider-anthropic", "config": {"model": "claude-haiku-3-5"}}],
        },
        "deep-specialist": {
            "description": "Thorough analysis (Opus - $15/1M)",
            "providers": [{"module": "provider-anthropic", "config": {"model": "claude-opus-4-1"}}],
        },
    }
}

print("Workflow:")
print("  1. User: 'Review codebase for issues'")
print("  2. Main: 'quick-scout, do fast scan'")
print("  3. Scout: 'Found 3 potential issues'")
print("  4. Main: 'deep-specialist, analyze these issues'")
print("  5. Specialist: 'Detailed analysis...'")
print("\nBenefit: Pay for Opus only when needed (60x savings on simple tasks)")

# Pattern 2: Parallel Delegation
print("\n" + "=" * 60)
print("Pattern 2: Parallel Delegation (Multiple agents simultaneously)\n")
print("Use case: Different aspects analyzed concurrently\n")

parallel_pattern = {
    "agents": {
        "security-auditor": {"description": "Security review"},
        "performance-analyzer": {"description": "Performance analysis"},
        "code-quality-checker": {"description": "Code quality"},
    }
}

print("Workflow:")
print("  Main: Spawn 3 agents in parallel")
print("     ├─ security-auditor")
print("     ├─ performance-analyzer")
print("     └─ code-quality-checker")
print("  All: Work independently")
print("  Main: Wait for all to complete")
print("  Main: Combine results")
print("\nBenefit: Faster overall completion (parallel execution)")

# Pattern 3: Chain of Responsibility
print("\n" + "=" * 60)
print("Pattern 3: Chain of Responsibility (Controlled depth)\n")
print("Use case: Specialized agent for sub-tasks\n")

chain_pattern = {
    "agents": {
        "general-engineer": {
            "description": "Can delegate to specialists",
            "tools": [
                {"module": "tool-filesystem"},
                {"module": "tool-task"},  # Can delegate
            ],
        },
        "database-expert": {
            "description": "Database specialist (end of chain)",
            "tools": [
                {"module": "tool-filesystem"}
                # NO tool-task
            ],
        },
    }
}

print("Workflow:")
print("  User → Main")
print("  Main → general-engineer (complex task)")
print("  general-engineer → database-expert (DB-specific part)")
print("  database-expert completes (cannot delegate further)")
print("  Results flow back: database-expert → general-engineer → Main → User")
print("\nBenefit: Depth-limited, specialized expertise")

# Pattern 4: Read-Only Analysis
print("\n" + "=" * 60)
print("Pattern 4: Read-Only Analysis (Safety through restriction)\n")
print("Use case: Safe exploration before making changes\n")

readonly_pattern = {
    "agents": {
        "safe-analyzer": {
            "description": "Can only read, cannot modify",
            "tools": [
                {
                    "module": "tool-filesystem",
                    "config": {"allowed_write_paths": []},  # NO writing
                }
            ],
        },
        "implementer": {
            "description": "Can modify files",
            "tools": [
                {"module": "tool-filesystem"},  # Full access
                {"module": "tool-bash"},
            ],
        },
    }
}

print("Workflow:")
print("  1. Main → safe-analyzer: 'Analyze the codebase'")
print("  2. safe-analyzer: Reads files, produces plan (cannot break anything)")
print("  3. Main reviews plan")
print("  4. Main → implementer: 'Execute this plan'")
print("  5. implementer: Makes changes")
print("\nBenefit: Safe exploration, controlled modification")

print("\n" + "=" * 60)
print("Pattern Selection Guide:\n")
print("  Progressive: Cost-sensitive, uncertain complexity")
print("  Parallel: Multiple independent aspects")
print("  Chain: Specialized expertise needed")
print("  Read-Only: Safety-critical operations")

## Part 8: Practical Example - Building a Development Team

In [ ]:
# Complete practical example: A development team configuration

dev_team_profile = {
    "profile": {
        "name": "dev-team",
        "version": "1.0.0",
        "description": "Complete development team with specialized agents",
    },
    "session": {"orchestrator": "loop-streaming", "context": "context-persistent"},
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"model": "claude-sonnet-4-5", "api_key": "${ANTHROPIC_API_KEY}"},
        }
    ],
    "tools": [
        {"module": "tool-filesystem"},
        {"module": "tool-bash"},
        {"module": "tool-web"},
        {"module": "tool-task"},  # Enable delegation
    ],
    "hooks": [{"module": "hooks-logging", "config": {"level": "INFO"}}],
    "agents": {
        # Senior architect for design decisions
        "architect": {
            "description": "Senior architect for system design (Opus)",
            "providers": [{"module": "provider-anthropic", "config": {"model": "claude-opus-4-1", "temperature": 0.2}}],
            "tools": [{"module": "tool-filesystem"}],
            "system": {"instruction": "System architect. Focus on scalable, maintainable designs."},
        },
        # Developer for implementation
        "developer": {
            "description": "Developer for implementation (Sonnet)",
            "providers": [
                {"module": "provider-anthropic", "config": {"model": "claude-sonnet-4-5", "temperature": 0.3}}
            ],
            "tools": [{"module": "tool-filesystem"}, {"module": "tool-bash"}],
            "system": {"instruction": "Developer. Implement clean, tested code."},
        },
        # QA engineer for testing
        "qa-engineer": {
            "description": "QA engineer for testing (Sonnet)",
            "providers": [
                {"module": "provider-anthropic", "config": {"model": "claude-sonnet-4-5", "temperature": 0.4}}
            ],
            "tools": [{"module": "tool-filesystem"}, {"module": "tool-bash"}],
            "system": {"instruction": "QA Engineer. Write comprehensive tests, find edge cases."},
        },
        # Code reviewer
        "code-reviewer": {
            "description": "Code reviewer for quality (Sonnet)",
            "providers": [
                {"module": "provider-anthropic", "config": {"model": "claude-sonnet-4-5", "temperature": 0.2}}
            ],
            "tools": [{"module": "tool-filesystem"}],
            "system": {"instruction": "Code reviewer. Check for bugs, style, best practices."},
        },
        # Documentation specialist
        "tech-writer": {
            "description": "Technical writer for docs (Sonnet)",
            "providers": [
                {"module": "provider-anthropic", "config": {"model": "claude-sonnet-4-5", "temperature": 0.6}}
            ],
            "tools": [{"module": "tool-filesystem"}, {"module": "tool-web"}],
            "system": {"instruction": "Technical writer. Clear, comprehensive documentation."},
        },
        # Quick helper for simple tasks
        "assistant": {
            "description": "General assistant for simple tasks (Haiku)",
            "providers": [
                {"module": "provider-anthropic", "config": {"model": "claude-haiku-3-5", "temperature": 0.5}}
            ],
            "tools": [{"module": "tool-filesystem"}],
            "system": {"instruction": "Assistant. Handle simple, routine tasks quickly."},
        },
    },
    "system_instruction": """
You are a senior engineering lead managing a development team.

Your team:
- architect: System design and architecture
- developer: Feature implementation
- qa-engineer: Test writing and quality
- code-reviewer: Code review and quality checks
- tech-writer: Documentation
- assistant: Simple tasks

Delegate appropriately based on task complexity and type.
""",
}

print("Complete Development Team Configuration")
print("=" * 60)
print(f"\nProfile: {dev_team_profile['profile']['name']}")
print(f"Description: {dev_team_profile['profile']['description']}\n")

print("Team Members:")
print("-" * 60)
for agent_name, agent_config in dev_team_profile["agents"].items():
    model = agent_config["providers"][0]["config"]["model"]
    temp = agent_config["providers"][0]["config"]["temperature"]
    print(f"\n{agent_name}:")
    print(f"  Role: {agent_config['description']}")
    print(f"  Model: {model}")
    print(f"  Temperature: {temp}")
    print(f"  Tools: {', '.join(t['module'] for t in agent_config['tools'])}")

print("\n" + "=" * 60)
print("Usage Examples:\n")

examples = [
    ("Design new API", "architect", "System design needed"),
    ("Implement login feature", "developer", "Code implementation"),
    ("Write tests for auth", "qa-engineer", "Testing needed"),
    ("Review pull request", "code-reviewer", "Quality check"),
    ("Update README", "tech-writer", "Documentation"),
    ("Count Python files", "assistant", "Simple task"),
]

for task, agent, reason in examples:
    print(f"Task: '{task}'")
    print(f"  → Delegate to: {agent}")
    print(f"  → Reason: {reason}")
    print()

# Save as YAML for use
yaml_output = f"""
# Save this as: profiles/dev-team.md
---
{json.dumps(dev_team_profile, indent=2)}
---
{dev_team_profile["system_instruction"]}
"""

print("=" * 60)
print("To use this configuration:\n")
print("1. Save to: profiles/dev-team.md")
print("2. Run: amplifier profile set dev-team")
print("3. Use: amplifier run 'Build authentication system'")
print("\nThe main agent will delegate to team members as needed!")

## Part 9: Testing Agent Configuration

In [ ]:
# Tools for testing and validating agent configurations


def validate_agent_config(config: dict[str, Any]) -> dict[str, Any]:
    """Comprehensive validation of agent configuration."""
    errors = []
    warnings = []
    info = []

    # Check required fields
    if "session" not in config:
        errors.append("Missing 'session' configuration")
    else:
        if "orchestrator" not in config["session"]:
            errors.append("Missing 'session.orchestrator'")
        if "context" not in config["session"]:
            errors.append("Missing 'session.context'")

    if "providers" not in config or not config["providers"]:
        errors.append("At least one provider required")

    # Check tool-task presence
    main_tools = [t["module"] for t in config.get("tools", [])]
    if "agents" in config and config["agents"] and "tool-task" not in main_tools:
        warnings.append("Agents defined but tool-task not in main tools (delegation won't work)")

    # Check each agent
    for agent_name, agent_config in config.get("agents", {}).items():
        # Check recursion risk
        agent_tools = [t["module"] for t in agent_config.get("tools", [])]
        if "tool-task" in agent_tools:
            warnings.append(f"Agent '{agent_name}' has tool-task (recursion risk!)")

        # Check description
        if "description" not in agent_config:
            warnings.append(f"Agent '{agent_name}' missing description")

        # Check system instruction
        if "system" not in agent_config or "instruction" not in agent_config.get("system", {}):
            warnings.append(f"Agent '{agent_name}' missing system instruction")

        # Info about model
        if "providers" in agent_config:
            model = agent_config["providers"][0]["config"].get("model", "unknown")
            info.append(f"Agent '{agent_name}' uses {model}")

    return {"valid": len(errors) == 0, "errors": errors, "warnings": warnings, "info": info}


# Test the dev team configuration
print("Agent Configuration Validation")
print("=" * 60)
print("\nValidating dev-team configuration...\n")

# Convert the profile to a mount plan format
test_config = {
    "session": dev_team_profile["session"],
    "providers": dev_team_profile["providers"],
    "tools": dev_team_profile["tools"],
    "agents": dev_team_profile["agents"],
}

result = validate_agent_config(test_config)

if result["valid"]:
    print("✅ Configuration is VALID\n")
else:
    print("❌ Configuration has ERRORS\n")

if result["errors"]:
    print("Errors:")
    for error in result["errors"]:
        print(f"  ❌ {error}")
    print()

if result["warnings"]:
    print("Warnings:")
    for warning in result["warnings"]:
        print(f"  ⚠️  {warning}")
    print()

if result["info"]:
    print("Info:")
    for item in result["info"]:
        print(f"  ℹ️  {item}")
    print()

# Test an invalid configuration
print("=" * 60)
print("Testing invalid configuration...\n")

invalid_config = {
    "session": {"orchestrator": "loop-streaming"},  # Missing context
    "tools": [{"module": "tool-filesystem"}],  # Missing tool-task
    "agents": {
        "bad-agent": {
            "tools": [
                {"module": "tool-filesystem"},
                {"module": "tool-task"},  # Recursion risk!
            ]
        }
    },
}

result2 = validate_agent_config(invalid_config)

if result2["errors"]:
    print("Errors:")
    for error in result2["errors"]:
        print(f"  ❌ {error}")
    print()

if result2["warnings"]:
    print("Warnings:")
    for warning in result2["warnings"]:
        print(f"  ⚠️  {warning}")

print("\n" + "=" * 60)
print("Validation Checklist:\n")
print("  ✓ session.orchestrator and session.context present")
print("  ✓ At least one provider configured")
print("  ✓ tool-task in main tools (if using agents)")
print("  ✓ NO tool-task in sub-agents (prevent recursion)")
print("  ✓ All agents have descriptions")
print("  ✓ All agents have system instructions")

## Summary

This notebook demonstrated:

1. **Basic Configuration**: How agents inherit and override parent config
2. **Delegation Simulation**: How tool-task spawns sub-agents
3. **Multiple Agents**: Building a specialized agent team
4. **Cost Analysis**: Impact of model selection (60x difference!)
5. **Recursion Prevention**: Critical safety measure
6. **Configuration Inheritance**: Deep dive into merge behavior
7. **Common Patterns**: Progressive, parallel, chain, read-only
8. **Practical Example**: Complete dev team configuration
9. **Validation**: Tools to test configurations

### Key Takeaways

- **Agents enable specialization** - right tool for the job
- **Model selection is critical** - Haiku vs Opus is 60x cost difference
- **Always prevent recursion** - remove tool-task from sub-agents
- **Inheritance is powerful** - override only what you need
- **Patterns solve problems** - progressive, parallel, chain, read-only
- **Validate configurations** - catch issues before runtime

### Next Steps

1. Create your own agent team configuration
2. Start with progressive delegation (cheap → expensive)
3. Add specialized agents as needs grow
4. Monitor costs and optimize model selection
5. Use validation tools to catch configuration issues

### Related Documentation

- [Mount Plans Guide](./guides/mounts.md) - Agent configuration in mount plans
- [Profiles Guide](./guides/profiles.md) - Defining agents in profiles
- [Tools Guide](./guides/tools.md) - tool-task enables delegation
- [Modules Guide](./guides/modules.md) - Module system overview